In [10]:
import gdal
import shapely
import geopandas as gpd
import os
from functools import partial

In [11]:
exposure_to_curve = {}
with open('VulFl_Europe.dat', 'r') as f:
    for l in f:
        l = l.strip()
        if ',' in l:
            exposure, _, curve = l.split(',')
            exposure_to_curve[exposure] = curve

In [12]:
def get_curve_and_save(f, r):
    k = r.SE_INUNDA
    if k is None or k not in exposure_to_curve:
        missing_string = f'Cpx{r.CPX}_{r.USE_SECTOR}_{r.BS_TYPE}_{r.RES_TYPE}, {r.VALFIS}\n'
        #print(missing_string)
        f.write(str(missing_string))
        return None
        
    return exposure_to_curve[k]


In [13]:
shps_dir = '/Users/silvia/Desktop/Copernicus_Buna/GAR/gar_merged'
shps_out = '/Users/silvia/Desktop/Copernicus_Buna/GAR/gar_merged_curve'

shps = os.listdir(shps_dir)
shps = filter(lambda s:s.endswith('.shp'), shps)

for s in list(shps)[-2:]:
    c = 0
    print(s)
    fin = os.path.join(shps_dir, s)
    fout = os.path.join(shps_out, s)    
    missing_f = open(s[:-4]+'.txt', 'w', encoding='utf-8')
    get_curve = partial(get_curve_and_save, missing_f)
    df = gpd.read_file(fin)   
    df['curve'] = df.apply (lambda r: get_curve(r), axis=1)
    df.to_file(fout)
    missing_f.close()

gar_merged.shp
